In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [ ]:
# creating a db interface using connection string 

dbcon_string = "mysql://<user>:<password>@localhost:<port>"
db = create_engine(dbcon_string)

<b>Q1</b>: Some of the facilities charge a fee to members, but some do not.
Please list the names of the facilities that do.

In [4]:
# sql query
stmt = 'SELECT * FROM country_club.facilities WHERE membercost=0'

## with context manager, connect to the db and execute the sql
with db.connect() as con:
    resultProxy = con.execute(stmt)

# results of the sql execution as list of tuples
resultSet = resultProxy.fetchall()
resultSet

[('2', 'Badminton Court', 0.0, 15.5, 4000, 50),
 ('3', 'Table Tennis', 0.0, 5.0, 320, 10),
 ('7', 'Snooker Table', 0.0, 5.0, 450, 15),
 ('8', 'Pool Table', 0.0, 5.0, 400, 15)]

<b>Q10</b>: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!


In [6]:
# multi-line string

stmt = '''
    select facility_name, sum(cost) as revenue
    from
        (select
              fac.name as facility_name
            , concat(mem.firstname, " ", mem.surname) as member_name
            , case when mem.memid=0 then (fac.guestcost * 2 * bkg.slots)
                    else (fac.membercost * 2 * bkg.slots)
                end as cost
        from
              country_club.bookings bkg
            , country_club.members mem
            , country_club.facilities fac
        where
                bkg.memid = mem.memid
            and bkg.facid = fac.facid
        ) raw
    group by facility_name
    having sum(cost)<1000
    order by sum(cost)
'''
## with context manager, connect to the db and execute the sql
with db.connect() as con:
    resultProxy = con.execute(stmt)

# results of the sql execution in a dataframe
resultSet = resultProxy.fetchall()
pd.DataFrame(resultSet, columns=resultProxy.keys())

,facility_name,revenue
0,Table Tennis,360.0
1,Snooker Table,480.0
2,Pool Table,540.0
